#### Creación base de datos - SQL
Este notebook se va a utilizar para crear y rellenar la base de datos con las tablas previamente trabajadas.

In [1]:
import mysql.connector as conn
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

In [2]:
with open('contraseñas.txt') as file:
    mypasswd = file.read()

In [3]:
# Nos conectamos al servidor 
conexion = conn.connect(host='localhost',
                        user='root',
                        passwd=f'{mypasswd}'
                        )

cursor  = conexion.cursor()

cursor

In [5]:
cursor.execute('drop database if exists transito')

cursor.execute('create database transito;')

In [6]:
conexion = conn.connect(host='localhost',
                        user='root',
                        passwd=f'{mypasswd}',
                        database='transito'
                        )

cursor  = conexion.cursor()

In [7]:
c = cursor.execute
c

<bound method CMySQLCursor.execute of <mysql.connector.cursor_cext.CMySQLCursor object at 0x000001E91808B820>>

#### Insertamos tablas en el database

In [34]:
# Leemos los 4 archivos csv que nos interesan
df_transit_clean = pd.read_csv('../data/transit_clean.csv')
df_puertos = pd.read_csv('../data/puertos.csv')
df_transit_plus_weather = pd.read_csv('../data/transit_plus_weather.csv')
df_otle_gt = pd.read_csv('../data/otle_gt.csv')

#### Transit clean

In [19]:
df_transit_clean.head()

,transit_id,barco,origen,destino,llegada,salida,alineacion,consignatorio,gt,escala,bandera,eslora,calado
0,1,VALIANT LADY,CANNES,IBIZA,31/05/2023 17:44,01/06/2023 18:15,ALIN.NORTE DE LA PLATAFORMA (del 22 al 2),ESTELA SHIPPING PALMA SA,108.192,192.0,BAHAMAS,277.0,8.0
1,2,ELEANOR ROOSEVELT,IBIZA,IBIZA,31/05/2023 22:28,01/06/2023 07:51,MUELLE PARAIRES (del 11 al 2),BALEARIA EUROLINEAS MARITIMAS S.A.U.,12.262,2066.0,CHIPRE,123.0,4.0
2,3,TENACIA,BARCELONA,BARCELONA,01/06/2023 04:23,01/06/2023 09:59,M. 1ª ALIN. PONIENTE SUR (del 14 al 1),TRANSPORTES Y CONSIGNACIONES MARITIMAS BALEAR SAU,25.993,2462.0,ITALIA,199.0,6.0
3,4,HEDY LAMARR,Valencia,Valencia,01/06/2023 04:53,01/06/2023 10:21,ALIN.OESTE DE LA PLATAFORMA (del 6 al 1),BALEARIA EUROLINEAS MARITIMAS S.A.U.,26.375,2422.0,CHIPRE,186.0,7.0
4,5,HYPATIA DE ALEJANDRIA,BARCELONA,BARCELONA,01/06/2023 05:19,01/06/2023 11:35,MUELLE DE LA 2 ALIN. DEL DIQUE DEL OESTE (del ...,BALEARIA EUROLINEAS MARITIMAS S.A.U.,28.658,2181.0,CHIPRE,187.0,7.0


In [26]:
c('drop table if exists trnasit_clean;')

query ='''

create table trnasit_clean(
    transit_id int primary key not null,
    barco varchar(45),
    origen varchar(45),
    destino	varchar(45),
    llegada	varchar(45),
    salida varchar(45),
    alineacion varchar(100),
    consignatorio varchar(100),
    gt float,
    escala float,
    bandera varchar(45),
    eslora float,
    calado float
    );
'''

c(query)

In [27]:
# Relleno datos
nombre_tabla = 'trnasit_clean'
nombre_columnas = ','.join(df_transit_clean.columns) # creo una string

#bucle por filas
for i in range(df_transit_clean.shape[0]):
    valores = tuple(df_transit_clean.iloc[i].values)

    insert_query = f'insert into {nombre_tabla} ({nombre_columnas}) values {valores}'

    cursor.execute(insert_query)


In [28]:
insert_query

"insert into trnasit_clean (transit_id,barco,origen,destino,llegada,salida,alineacion,consignatorio,gt,escala,bandera,eslora,calado) values (1049, 'ELEANOR ROOSEVELT', 'IBIZA', 'IBIZA', '31/08/2023 22:25', '01/09/2023 07:50', 'MUELLE PARAIRES (del 11 al 1)', 'BALEARIA EUROLINEAS MARITIMAS S.A.U.', 12.262, 3106.0, 'CHIPRE', 123.0, 4.0)"

In [29]:
# Comprobamos

cursor.execute('select * from trnasit_clean limit 2;')

for x in cursor:
    print(x)

(1, 'VALIANT LADY', 'CANNES', 'IBIZA', '31/05/2023 17:44', '01/06/2023 18:15', 'ALIN.NORTE DE LA PLATAFORMA (del 22 al 2)', 'ESTELA SHIPPING PALMA SA', 108.192, 192.0, 'BAHAMAS', 277.0, 8.0)
(2, 'ELEANOR ROOSEVELT', 'IBIZA', 'IBIZA', '31/05/2023 22:28', '01/06/2023 07:51', 'MUELLE PARAIRES (del 11 al 2)', 'BALEARIA EUROLINEAS MARITIMAS S.A.U.', 12.262, 2066.0, 'CHIPRE', 123.0, 4.0)


In [30]:
# Comprobamos tamaño
# check 

cursor.execute('select count(*) from trnasit_clean;')

for x in cursor:
    print(x)

(1049,)


#### Puertos

In [36]:
df_puertos = df_puertos.rename(columns={'transit_coords_id': 'puertos_id'})
df_puertos.head()

,puertos_id,Puerto,Latitud,Longitud
0,0,IBIZA,38.912653,1.442283
1,1,BARCELONA,41.346387,2.165532
2,2,Valencia,39.467328,-0.390288
3,3,"MAHON, MENORCA",39.904400,4.247600
4,4,SAGUNTO,39.783600,-0.210800


In [42]:
c('drop table if exists puertos;')

query ='''

create table puertos(
    puertos_id int primary key not null,
    Puerto varchar(100),
    Latitud float,
    Longitud float
    );
'''

c(query)

In [43]:
# Relleno datos
nombre_tabla = 'puertos'
nombre_columnas = ','.join(df_puertos.columns) # creo una string

#bucle por filas
for i in range(df_puertos.shape[0]):
    valores = tuple(df_puertos.iloc[i].values)

    insert_query = f'insert into {nombre_tabla} ({nombre_columnas}) values {valores}'

    cursor.execute(insert_query)

In [44]:
insert_query

"insert into puertos (puertos_id,Puerto,Latitud,Longitud) values (42, 'PORT EVERGLADES', 26.166667, -80.166667)"

In [45]:
# Comprobamos

cursor.execute('select * from puertos limit 2;')

for x in cursor:
    print(x)

(0, 'IBIZA', 38.9127, 1.44228)
(1, 'BARCELONA', 41.3464, 2.16553)


In [46]:
# Comprobamos tamaño
# check 

cursor.execute('select count(*) from puertos;')

for x in cursor:
    print(x)

(43,)
